In [1]:
import pandas as pd
import collections
import etl_idesp
pd.set_option('display.max_columns', None)

In [2]:
def aponta_diferencas(lista_escolas):
    n_escolas = len(ESCOLAS_FATO)
    exemplos_diferentes = set(ESCOLAS_FATO)
    print('Escolas da tabela fato: ', n_escolas)
    for item in lista_escolas:
        diferencas_fato = ESCOLAS_FATO.difference(item[1])
        exemplos_diferentes = exemplos_diferentes.intersection(diferencas_fato)
        print('Base: ', item[0], 
              ' escolas: ', len(item[1]), 
              ' faltam do fato: ', len(diferencas_fato)/n_escolas)

    print('Nao encontrado em nenhuma outra base: ', len(exemplos_diferentes))
    if len(exemplos_diferentes) > 5:
        exemplos_diferentes = list(exemplos_diferentes)[0:5]
    else:
        exemplos_diferentes = list(exemplos_diferentes)[0:len(diferencas_fato)]
    print('Exemplos nao encontrados: ', exemplos_diferentes)


# OBJETIVO
- Apontar os joins que nao puderam ser feitos na tabela fato.
- Listar as caracteristicas que definem a escola e que podem ser relevantes para os estudos.

In [ ]:
df_idesp = etl_idesp.etl()
df_idesp.id_escola = df_idesp.id_escola.astype(int)

In [ ]:
df_idesp.head(3)

In [ ]:
ESCOLAS_FATO = set(df_idesp.id_escola)

# Dimensao escola (associada a uma dada gestao)
- Escola fisica.
- Classes.
- Idesp: Nome diretoria.
- Endereco.
- Dependencias fisicas

In [6]:
arquivo = 'D:\\DESAFIO_SEDUC\\dados-VERSAO_1\\instalacao_fisicas\\06_Escolas_Dependencias.csv'
dep = pd.read_csv(arquivo, sep=';')

In [7]:
dep.CODESC = dep.CODESC.astype(int)
escolas_dep = set(dep.CODESC)

In [8]:
dep = dep.rename({'CODESC': 'id_escola'}, axis=1)

In [9]:
dep = dep.drop([
    'NOMEDEP', # todas sao 'ESTADUAL - SE'
    'SITUACAO', # todos ativos
    'DE' # diretoria vai vir da tabela idesp
],axis=1)
# DE A diretoria vai vir da tabela idesp


In [10]:
escolas_dep = set(dep.id_escola)

In [11]:
aponta_diferencas([['dependencias', escolas_dep]])

Escolas da tabela fato:  5410
Base:  dependencias  escolas:  5608  faltam do fato:  0.06451016635859519
Nao encontrado em nenhuma outra base:  349
Exemplos nao encontrados:  [43011, 47119, 32785, 30739, 901143]


# Dimensao endereco

In [12]:
arquivo = 'D:\\DESAFIO_SEDUC\\dados-VERSAO_1\\escolas_enderecos\\11_escolas_enderecos_0.csv'
with open(arquivo, encoding="utf8") as file_in:
    lines = []
    for line in file_in:
        lines.append(line)
dados_lines = []
len_lines = len(lines[0].split(','))
for line in lines[1:]:
    line = line.rstrip().split(',')
    if len(line) == len_lines:  # linhas com numero incorreto de colunas foram descartadas (1)
        dados_lines.append(line)
end = pd.DataFrame(data=dados_lines)
end.columns = lines[0].split(',')
end.head(3)

,nomedep,depadm,mun,codmun,de,CD_ESCOLA,CD_DIRETORIA,CD_DIRETORIA_ESTADUAL,CD_DIRETORIA_SUPVS_PROPR,NM_COMPLETO_ESCOLA,CD_UNIDADE,DS_ENDERECO,COMPLEMENTO,NUMERO,CD_ORIGEM_UNIDADE,CD_ORIGEM_ESCOLA,CD_ORIGEM_ENDERECO,LATITUDE,LONGITUDE
0,ESTADUAL - SE,1,SAO PAULO,100,NORTE 1,36444,10101,10101,10101,GENESIO DE ALMEIDA MOURA DOUTOR,37806,DOMINGOS AREVALO,RUA,862,37806,36444,37866,-23.447,-46.6967
1,ESTADUAL - SE,1,SAO PAULO,100,LESTE 5,1582,10205,10205,10205,DOMINGOS FAUSTINO SARMIENTO,24902,21 DE ABRIL,RUA,970,24902,1582,24962,-23.5422317000605,-46.6057931815944
2,ESTADUAL - SE,1,SAO PAULO,100,LESTE 5,2173,10205,10205,10205,JOAO BORGES PROFESSOR,24901,ITAPURA,RUA,976,24901,2173,24961,-23.5473,-46.566


In [13]:
end = end.rename({'CD_ESCOLA': 'id_escola'}, axis=1)
end.id_escola = end.id_escola.astype(int)
escolas_end = set(end.id_escola)

In [14]:
aponta_diferencas([
    ['dependencias', escolas_dep],
    ['enderecos',escolas_end]
])

Escolas da tabela fato:  5410
Base:  dependencias  escolas:  5608  faltam do fato:  0.06451016635859519
Base:  enderecos  escolas:  27328  faltam do fato:  0.05563770794824399
Nao encontrado em nenhuma outra base:  293
Exemplos nao encontrados:  [43011, 39433, 21003, 18958, 47119]


In [15]:
arquivo = 'D:\\DESAFIO_SEDUC\\dados-VERSAO_1\\escolas_enderecos\\11_Escolas_Coordenadas.csv'
with open(arquivo, encoding="latin-1") as file_in:
    lines = []
    for line in file_in:
        lines.append(line)
dados_lines = []
len_lines = len(lines[0].split(';'))
for line in lines[1:]:
    line = line.rstrip().split(';')
    if len(line) == len_lines:  # linhas com numero incorreto de colunas foram descartadas (1)
        dados_lines.append(line)
end2 = pd.DataFrame(data=dados_lines)
end2.columns = lines[0].split(';')
end2.head(3)

,NOMEDEP,DE,MUN,DISTR,COD_ESC,NOMESC,SITUACAO,TIPOESC,ENDESC,NUMESC,COMPLEND,CEP,BAIESC,ZONA,DS_LONGITUDE,DS_LATITUDE,CODVINC
0,ESTADUAL - SE,NORTE 1,SAO PAULO,JAGUARA,12,AYRES DE MOURA PROFESSOR,Ativa,EE,RUA ARTUR ORLANDO,907,RUA,5118000,VILA JAGUARA,URBANA,"-46,74909973","-23,5121994",
1,ESTADUAL - SE,NORTE 1,SAO PAULO,PERUS,24,GAVIAO PEIXOTO BRIGADEIRO,Ativa,EE,MOGEIRO,710,RUA,5208230,VILA PERUS,URBANA,"-46,75939941","-23,40469933",
2,ESTADUAL - SE,NORTE 1,SAO PAULO,FREGUESIA DO O,36,JOAO SOLIMEO,Ativa,EE,DO SABAO,1497,ESTRADA,2806000,JARDIM MARISTELA,URBANA,"-46,70082092","-23,48127937",


In [16]:
len(end)

28729

In [17]:
len(end2)

5736

In [18]:
end2 = end2.rename({'COD_ESC': 'id_escola'}, axis=1)
end2.id_escola = end2.id_escola.astype(int)
escolas_end2 = set(end2.id_escola)

In [19]:
aponta_diferencas([
    ['dependencias', escolas_dep],
    ['enderecos',escolas_end],
    ['coordenadas',escolas_end2]
])

Escolas da tabela fato:  5410
Base:  dependencias  escolas:  5608  faltam do fato:  0.06451016635859519
Base:  enderecos  escolas:  27328  faltam do fato:  0.05563770794824399
Base:  coordenadas  escolas:  5736  faltam do fato:  0.06395563770794824
Nao encontrado em nenhuma outra base:  293
Exemplos nao encontrados:  [43011, 39433, 21003, 18958, 47119]


# Cluster

In [20]:
arquivo = 'D:\\DESAFIO_SEDUC\\dados-VERSAO_1\\cluster_de_escolas_similares\\12_MMR_Clusters.csv'
cluster = pd.read_csv(arquivo, sep=';')
cluster.CD_ESCOLA = cluster.CD_ESCOLA.astype(int)
cluster = cluster.rename({'CD_ESCOLA': 'id_escola'}, axis=1)

In [21]:
escolas_clu = set(cluster.id_escola)

In [22]:
aa = aponta_diferencas([
    ['dependencias', escolas_dep],
    ['enderecos',escolas_end],
    ['coordenadas',escolas_end2],
    ['cluster', escolas_clu]    
])

Escolas da tabela fato:  5410
Base:  dependencias  escolas:  5608  faltam do fato:  0.06451016635859519
Base:  enderecos  escolas:  27328  faltam do fato:  0.05563770794824399
Base:  coordenadas  escolas:  5736  faltam do fato:  0.06395563770794824
Base:  cluster  escolas:  5056  faltam do fato:  0.06598890942698706
Nao encontrado em nenhuma outra base:  293
Exemplos nao encontrados:  [43011, 39433, 21003, 18958, 47119]


# Dimensao classes

In [23]:
arquivo = 'dados-VERSAO_1\matriculas\/10_Escolas_Classes_Qtde_Alunos.csv'
classes = pd.read_csv(arquivo, sep=';')

In [24]:
classes = classes.rename({'COD_ESC': 'id_escola'}, axis=1)
classes.id_escola = classes.id_escola.astype(int)

In [25]:
escolas_classes = set(classes.id_escola)

In [26]:
aponta_diferencas([
    ['dependencias', escolas_dep],
    ['enderecos',escolas_end],
    ['coordenadas',escolas_end2],
    ['cluster', escolas_clu],
    ['classes', escolas_classes]
])

Escolas da tabela fato:  5410
Base:  dependencias  escolas:  5608  faltam do fato:  0.06451016635859519
Base:  enderecos  escolas:  27328  faltam do fato:  0.05563770794824399
Base:  coordenadas  escolas:  5736  faltam do fato:  0.06395563770794824
Base:  cluster  escolas:  5056  faltam do fato:  0.06598890942698706
Base:  classes  escolas:  5638  faltam do fato:  0.06395563770794824
Nao encontrado em nenhuma outra base:  293
Exemplos nao encontrados:  [43011, 39433, 21003, 18958, 47119]


# CARGA HORARIA

In [27]:
arquivo = 'D:\\DESAFIO_SEDUC\\dados-VERSAO_1\\carga-horria-por-docente-All-2020-04-13_1010\\BASE_CARGA_HOR_SALA_AULA_0419.csv'
hora = pd.read_csv(arquivo, sep=';')

In [28]:
hora.head(3)

,REGIAO,NOMEDE,CD_INEP,CODESC,CATESC,TPESC,UA,NOMEESC,NM_MUNICIPIO,MODAL,NOME,RG12,DI,CPF,CARGO_C,NM_CARGOC,CODMAE,DEN_CODMAE,CICLO,MATERIA,DEN_MATERIA,TOT_AULA_LIVRE,TOT_AULA_SUBST,TOT_GERAL_AULA,TOT_AULA_LIVRE_NOTURNO,TOT_AULA_SUBST_NOTURNO,TOT_AULAS_NOTURNO,JORNADA,id_interno
0,INTERIOR,D.E.REG. LINS,35026141,26141,1,8,43829,HUGO GAMBETTI PROF,PROMISSAO,EF INICIAIS + FINAIS + EM,318519,318519,1,318519,6409,PROFESSOR EDUCACAO BASICA II,1100,LING.PORTUGUESA,CII,1100,LING.PORTUGUESA,24,0,24,0,0,0,C,318519
1,INTERIOR,D.E.REG. LINS,35026104,26104,1,8,43828,ORLANDO DONDA PROF,PROMISSAO,EF FINAIS + EM,184802,184802,1,184802,6409,PROFESSOR EDUCACAO BASICA II,2700,MATEMATICA,CII,2700,MATEMATICA,24,0,24,0,0,0,B,184802
2,INTERIOR,D.E.REG. SANTO ANASTACIO,35032232,32232,1,8,44687,ANTONIO DE CARVALHO LEITAO PROFESSOR,PRESIDENTE EPITACIO,EF FINAIS + EM,318530,318530,1,318530,6409,PROFESSOR EDUCACAO BASICA II,2800,QUIMICA,EM,2800,QUIMICA,2,0,2,0,0,0,,318530


In [29]:
hora = classes.rename({'CODESC': 'id_escola'}, axis=1)
hora.id_escola = hora.id_escola.astype(int)

In [30]:
escolas_hora = set(hora.id_escola)

In [31]:
aponta_diferencas([
    ['dependencias', escolas_dep],
    ['enderecos',escolas_end],
    ['coordenadas',escolas_end2],
    ['classes', escolas_classes],
])

Escolas da tabela fato:  5410
Base:  dependencias  escolas:  5608  faltam do fato:  0.06451016635859519
Base:  enderecos  escolas:  27328  faltam do fato:  0.05563770794824399
Base:  coordenadas  escolas:  5736  faltam do fato:  0.06395563770794824
Base:  classes  escolas:  5638  faltam do fato:  0.06395563770794824
Nao encontrado em nenhuma outra base:  293
Exemplos nao encontrados:  [43011, 39433, 21003, 18958, 47119]


In [32]:
293/5410

0.054158964879852126

# Dimensao Servidores (associados a uma dada gestao)
- A dimensao servidores vai ter que ter um id_escola.
- Servidores da escola.
- Servidores ativos: Vou colocar os servidores na escola correspondente.
- Atribuicao digital: Disciplinas sem docente.
- Ausencias por servidor: Numero total de ausencias da escola.
- Carga horaria.
- Cluster.

# CLUSTER
- CD_CLUSTERS e o id
- CD_DIRETORIA ha 91 diretorias
- Numero do cluster: 9
- NR_CICLO: 3
- CD_GRUPO_CLUSTER

In [6]:
arquivo = 'D:\\DESAFIO_SEDUC\\dados-VERSAO_1\\cluster_de_escolas_similares\\12_MMR_Clusters.csv'
cluster = pd.read_csv(arquivo, sep=';')
cluster.CD_ESCOLA = cluster.CD_ESCOLA.astype(int)
cluster = cluster.rename({'CD_ESCOLA': 'id_escola'}, axis=1)

In [7]:
clu = {}
clu['id_escola'] = []
clu['nr_cluster'] = []
for nome, dados in cluster.groupby('id_escola'):
    clu['id_escola'].append(nome)
    clu['nr_cluster'].append(dados.NR_CLUSTER.max())  


In [8]:
clu = pd.DataFrame(clu)

In [9]:
esc_cl = pd.merge(esc, clu, on='id_escola',how='left')

In [10]:
esc_cl.loc[:,['nota','nr_cluster']].corr()

,nota,nr_cluster
nota,1.000000,-0.220918
nr_cluster,-0.220918,1.000000


In [11]:
esc_cl.nr_cluster.describe()

count    2134.000000
mean        4.126054
std         2.022273
min         1.000000
25%         2.000000
50%         4.000000
75%         6.000000
max         9.000000
Name: nr_cluster, dtype: float64

In [30]:
esc_cl[esc_cl.nr_cluster.isin([1,2,3,4])].nota.mean()

4.40099133369823

In [39]:
esc_cl[esc_cl.nr_cluster.isin([1,2,3,4])].nota.count()

1218

In [31]:
esc_cl[esc_cl.nr_cluster.isin([5,9])].nota.mean()

4.121484203296703

In [40]:
esc_cl[esc_cl.nr_cluster.isin([5,9])].nota.count()

364

In [32]:
esc_cl[esc_cl.nr_cluster.isin([6,7,8])].nota.mean()

3.5821358192431565

In [41]:
esc_cl[esc_cl.nr_cluster.isin([6,7,8])].nota.count()

552

In [33]:
esc_cl[esc_cl.nr_cluster == 0].nota.mean()

2.776062744507975

In [38]:
esc_cl[esc_cl.nr_cluster == 0].count()

id_escola     3323
nota          3323
nr_cluster    3323
dtype: int64

# NUMERO DE CLASSES

In [30]:
arquivo = 'dados\matriculas\/10_Escolas_Classes_Qtde_Alunos.csv'
classes = pd.read_csv(arquivo, sep=';')

In [32]:
classes.COD_ESC = classes.COD_ESC.astype(int)

In [34]:
classes = classes.rename({'COD_ESC': 'id_escola'}, axis=1)

In [ ]:
result = pd.merge(esc_cl, )

In [36]:
classes.head(10)

,ANO,NOMEDEP,DE,MUN,DISTR,id_escola,NOMESC,TIPOESC,TIPOESC_DESC,SITUACAO,CLASSE,TIPOCLASSE,TIPOCLASSE_DESC,GRAU,TipoEnsino,SERIE,QTDE_ALUNOS,CODVINC
0,2014,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,30806,HELEN KELLER,8,EE,Ativa,182570713,0,CLASSE COMUM,1,ENSINO FUNDAMENTAL DE 8 ANOS,8,21,NaN
1,2014,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,30806,HELEN KELLER,8,EE,Ativa,182570762,0,CLASSE COMUM,2,ENSINO MEDIO,2,37,NaN
2,2014,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,30806,HELEN KELLER,8,EE,Ativa,186061859,1,CLASSE MULTISSERIADA,5,EJA ENSINO MÉDIO,10,25,NaN
3,2014,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,31045,DURVALINO GRION PROF,8,EE,Ativa,182570440,0,CLASSE COMUM,2,ENSINO MEDIO,2,26,NaN
4,2014,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,31045,DURVALINO GRION PROF,8,EE,Ativa,182570465,0,CLASSE COMUM,2,ENSINO MEDIO,3,33,NaN
5,2014,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,31045,DURVALINO GRION PROF,8,EE,Ativa,182570523,0,CLASSE COMUM,14,ENSINO FUNDAMENTAL DE 9 ANOS,7,30,NaN
6,2014,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,31112,FLEURIDES CAVALLINI MENECHINO PROFA,8,EE,Ativa,182570127,0,CLASSE COMUM,1,ENSINO FUNDAMENTAL DE 8 ANOS,7,28,NaN
7,2014,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,31112,FLEURIDES CAVALLINI MENECHINO PROFA,8,EE,Ativa,182570184,0,CLASSE COMUM,1,ENSINO FUNDAMENTAL DE 8 ANOS,8,35,NaN
8,2014,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,31112,FLEURIDES CAVALLINI MENECHINO PROFA,8,EE,Ativa,182570192,0,CLASSE COMUM,2,ENSINO MEDIO,1,35,NaN
9,2014,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,31112,FLEURIDES CAVALLINI MENECHINO PROFA,8,EE,Ativa,182570218,0,CLASSE COMUM,2,ENSINO MEDIO,1,32,NaN
